In [6]:
!dir ..\data

 Volume in drive C is OS
 Volume Serial Number is A2DE-672A

 Directory of C:\Users\William\Documents\Turing\cross_check\data

02/12/2019  12:57 PM    <DIR>          .
02/12/2019  12:57 PM    <DIR>          ..
02/12/2019  12:57 PM         1,072,631 game.csv
02/12/2019  12:57 PM             2,628 game_small.csv
02/12/2019  12:57 PM         1,133,359 game_teams_stats.csv
02/12/2019  12:57 PM             2,942 game_teams_stats_small.csv
02/12/2019  12:57 PM               424 game_teams_stats_very_small.csv
02/12/2019  12:57 PM               418 game_very_small.csv
02/12/2019  12:57 PM             1,953 team_info.csv
02/12/2019  12:57 PM               153 team_info_very_small.csv
               8 File(s)      2,214,508 bytes
               2 Dir(s)  54,012,911,616 bytes free


In [7]:
import pandas as pd

In [12]:
root = "../data/"
game = root+'game.csv'
small_game = root+'game_small.csv'
vs_game = root+'game_very_small.csv'

gt = root+'game_teams_stats.csv'
small_gt = root+'game_teams_stats_small.csv'
vs_gt = root+ 'game_teams_stats_very_small.csv'

team = root+'team_info.csv'



game_df = pd.read_csv(game)
small_game_df = pd.read_csv(small_game)
vs_game_df = pd.read_csv(vs_game)

gt_df = pd.read_csv(gt)
small_gt_df = pd.read_csv(small_gt)
vs_gt_df = pd.read_csv(vs_gt)

team_df = pd.read_csv(team)

In [16]:
gt_df.merge(game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]


,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,season,type,date_time,venue_time_zone_tz,teamName
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,3,0,44.8,17,7,20122013,P,2013-05-16,EDT,Rangers
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,5,0,51.7,1,4,20122013,P,2013-05-19,EDT,Rangers
2,2012030223,3,home,False,REG,John Tortorella,1,24,37,2,2,0,38.2,7,9,20122013,P,2013-05-21,EDT,Rangers
3,2012030224,3,home,True,OT,John Tortorella,4,32,40,8,4,1,46.3,9,7,20122013,P,2013-05-23,EDT,Rangers
4,2012030225,3,away,False,REG,John Tortorella,1,29,25,13,2,1,50.9,5,3,20122013,P,2013-05-25,EDT,Rangers
5,2012030131,3,away,False,REG,John Tortorella,1,36,41,12,4,0,48.4,7,9,20122013,P,2013-05-02,EDT,Rangers
6,2012030132,3,away,False,OT,John Tortorella,0,24,58,8,3,0,48.6,7,5,20122013,P,2013-05-04,EDT,Rangers
7,2012030133,3,home,True,REG,John Tortorella,4,30,33,6,6,1,47.4,13,11,20122013,P,2013-05-06,EDT,Rangers
8,2012030134,3,home,True,REG,John Tortorella,4,34,38,4,4,1,64.2,4,8,20122013,P,2013-05-08,EDT,Rangers
9,2012030135,3,away,False,OT,John Tortorella,1,25,43,4,4,0,41.7,14,9,20122013,P,2013-05-10,EDT,Rangers


In [15]:
to_keep = ['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'goals',
       'shots', 'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals',
       'faceOffWinPercentage', 'giveaways', 'takeaways', 'season', 'type',
       'date_time','venue_time_zone_tz','teamName']

In [17]:
full_df = gt_df.merge(game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]
small_df = small_gt_df.merge(small_game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]
vs_df = vs_gt_df.merge(vs_game_df, on = "game_id").merge(team_df, on = "team_id")[to_keep]

In [18]:
vs_df

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,season,type,date_time,venue_time_zone_tz,teamName
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,3,0,44.8,17,7,20122013,P,2013-05-16,EDT,Rangers
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,5,0,51.7,1,4,20122013,P,2013-05-19,EDT,Rangers
2,2012030221,6,home,True,OT,Claude Julien,3,48,51,6,4,1,55.2,4,5,20122013,P,2013-05-16,EDT,Bruins
3,2012030222,6,home,True,REG,Claude Julien,5,32,36,19,1,0,48.3,16,6,20122013,P,2013-05-19,EDT,Bruins


In [19]:
small_df

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,season,type,date_time,venue_time_zone_tz,teamName
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,3,0,44.8,17,7,20122013,P,2013-05-16,EDT,Rangers
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,5,0,51.7,1,4,20122013,P,2013-05-19,EDT,Rangers
2,2012030223,3,home,False,REG,John Tortorella,1,24,37,2,2,0,38.2,7,9,20122013,P,2013-05-21,EDT,Rangers
3,2012030224,3,home,True,OT,John Tortorella,4,32,40,8,4,1,46.3,9,7,20122013,P,2013-05-23,EDT,Rangers
4,2012030225,3,away,False,REG,John Tortorella,1,29,25,13,2,1,50.9,5,3,20122013,P,2013-05-25,EDT,Rangers
5,2012030221,6,home,True,OT,Claude Julien,3,48,51,6,4,1,55.2,4,5,20122013,P,2013-05-16,EDT,Bruins
6,2012030222,6,home,True,REG,Claude Julien,5,32,36,19,1,0,48.3,16,6,20122013,P,2013-05-19,EDT,Bruins
7,2012030223,6,away,True,REG,Claude Julien,2,34,28,6,0,0,61.8,10,7,20122013,P,2013-05-21,EDT,Bruins
8,2012030224,6,away,False,OT,Claude Julien,3,40,24,8,4,2,53.7,8,6,20122013,P,2013-05-23,EDT,Bruins
9,2012030225,6,home,True,REG,Claude Julien,3,32,35,11,3,1,49.1,12,9,20122013,P,2013-05-25,EDT,Bruins


In [33]:
df_names = ['full','small', 'vs']
dfs = [full_df,small_df, vs_df]

### highest_total_score

In [39]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('game_id')['goals'].sum().sort_values(ascending = False).head(1), '\n')

full 
 game_id
2016020661    15
Name: goals, dtype: int64 

small 
 game_id
2012030312    7
Name: goals, dtype: int64 

vs 
 game_id
2012030222    7
Name: goals, dtype: int64 



### lowest_total_score

In [38]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('game_id')['goals'].sum().sort_values(ascending = True).head(1), '\n')

full 
 game_id
2014020456    0
Name: goals, dtype: int64 

small 
 game_id
2012030314    1
Name: goals, dtype: int64 

vs 
 game_id
2012030221    5
Name: goals, dtype: int64 



### biggest_blowout

In [64]:

def find_diff(x):
    if len(x) == 1: return 0
    else: return abs(x[0]-x[1])
    
    
for name, df in zip(df_names, dfs):
    print(name,"\n",df.groupby('game_id')['goals'].unique().apply(find_diff).sort_values(ascending = False).head(1), "\n")

full 
 game_id
2016020158    10
Name: goals, dtype: int64 

small 
 game_id
2012030312    5
Name: goals, dtype: int64 

vs 
 game_id
2012030222    3
Name: goals, dtype: int64 



### count_of_games_by_season

In [42]:
for name, df in zip(df_names, dfs):
    print(name,"\n",df.groupby('season')['goals'].count()/2, "\n")

full 
 season
20122013     806.0
20132014    1323.0
20142015    1319.0
20152016    1321.0
20162017    1317.0
20172018    1355.0
Name: goals, dtype: float64 

small 
 season
20122013    20.0
Name: goals, dtype: float64 

vs 
 season
20122013    2.0
Name: goals, dtype: float64 



### percentage_visitor_wins

In [43]:
vs_df

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,season,type,date_time,venue_time_zone_tz,teamName
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,3,0,44.8,17,7,20122013,P,2013-05-16,EDT,Rangers
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,5,0,51.7,1,4,20122013,P,2013-05-19,EDT,Rangers
2,2012030221,6,home,True,OT,Claude Julien,3,48,51,6,4,1,55.2,4,5,20122013,P,2013-05-16,EDT,Bruins
3,2012030222,6,home,True,REG,Claude Julien,5,32,36,19,1,0,48.3,16,6,20122013,P,2013-05-19,EDT,Bruins


In [44]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "away"].won.mean(),"\n")

full 
 0.4504770864131165 

small 
 0.3 

vs 
 0.0 



### percentage_home_wins

In [45]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "home"].won.mean(),"\n")

full 
 0.5495229135868834 

small 
 0.7 

vs 
 1.0 



 ### average_goals_per_game

In [46]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('game_id')['goals'].sum().mean(),"\n")

full 
 5.44080096761188 

small 
 4.65 

vs 
 6.0 



### average_goals_by_season

In [53]:
for name, df in zip(df_names, dfs):
    print(name,"\n",df.groupby('season')['goals'].sum() /( df.groupby('season')['goals'].count()/2), "\n")

full 
 season
20122013    5.277916
20132014    5.361300
20142015    5.304776
20152016    5.331567
20162017    5.431283
20172018    5.863469
Name: goals, dtype: float64 

small 
 season
20122013    4.65
Name: goals, dtype: float64 

vs 
 season
20122013    6.0
Name: goals, dtype: float64 



# League Stats

### count_of_teams

In [65]:
for name, df in zip(df_names, dfs):
    print(name,"\n",df['team_id'].unique().shape[0],"\n")

full 
 32 

small 
 7 

vs 
 2 



### best_offense

In [70]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('teamName')['goals'].mean().sort_values(ascending = False).head(1), "\n")

full 
 teamName
Golden Knights    3.186275
Name: goals, dtype: float64 

small 
 teamName
Senators    3.5
Name: goals, dtype: float64 

vs 
 teamName
Bruins    4
Name: goals, dtype: int64 



### worst_offense

In [71]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('teamName')['goals'].mean().sort_values(ascending = True).head(1), "\n")

full 
 teamName
Sabres    2.220524
Name: goals, dtype: float64 

small 
 teamName
Penguins    0.5
Name: goals, dtype: float64 

vs 
 teamName
Rangers    2
Name: goals, dtype: int64 



### highest_scoring_home_team

In [73]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "home"].groupby("teamName")['goals'].mean().sort_values(ascending = False).head(1), "\n") 

full 
 teamName
Golden Knights    3.568627
Name: goals, dtype: float64 

small 
 teamName
Senators    4.5
Name: goals, dtype: float64 

vs 
 teamName
Bruins    4
Name: goals, dtype: int64 



### lowest_scoring_home_team

In [74]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "home"].groupby("teamName")['goals'].mean().sort_values(ascending = True).head(1), "\n") 

full 
 teamName
Sabres    2.248908
Name: goals, dtype: float64 

small 
 teamName
Penguins    0.5
Name: goals, dtype: float64 

vs 
 teamName
Bruins    4
Name: goals, dtype: int64 



### highest_scoring_visitor

In [75]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "away"].groupby("teamName")['goals'].mean().sort_values(ascending = False).head(1), "\n") 

full 
 teamName
Capitals    2.901515
Name: goals, dtype: float64 

small 
 teamName
Bruins    3.5
Name: goals, dtype: float64 

vs 
 teamName
Rangers    2
Name: goals, dtype: int64 



### lowest_scoring_visitor

In [76]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df[df.HoA == "away"].groupby("teamName")['goals'].mean().sort_values(ascending = True).head(1), "\n") 

full 
 teamName
Sabres    2.19214
Name: goals, dtype: float64 

small 
 teamName
Penguins    0.5
Name: goals, dtype: float64 

vs 
 teamName
Rangers    2
Name: goals, dtype: int64 



### winnigest_team

In [78]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby("teamName")['won'].mean().sort_values(ascending = False).head(1), "\n")

full 
 teamName
Golden Knights    0.627451
Name: won, dtype: float64 

small 
 teamName
Bruins    0.888889
Name: won, dtype: float64 

vs 
 teamName
Bruins    True
Name: won, dtype: bool 



# Need to check below

### best_fans

In [80]:
for name, df in zip(df_names, dfs):
    home_win = df[df.HoA == "home"].groupby("teamName")['won'].mean().sort_index()
    away_win = df[df.HoA == "away"].groupby("teamName")['won'].mean().sort_index()
    
    print(name, "\n", (home_win-away_win).sort_values(ascending = False).head(1), "\n")

full 
 teamName
Flyers    0.18229
Name: won, dtype: float64 

small 
 teamName
Senators    0.5
Name: won, dtype: float64 

vs 
 teamName
Bruins    NaN
Name: won, dtype: object 



### worst_fans

In [83]:
for name, df in zip(df_names, dfs):
    home_win = df[df.HoA == "home"].groupby("teamName")['won'].mean().sort_index()
    away_win = df[df.HoA == "away"].groupby("teamName")['won'].mean().sort_index()
    
    away_minus_home = (away_win-home_win)
    
    print(name, "\n", away_minus_home[away_minus_home <0], "\n")

full 
 teamName
Avalanche        -0.125442
Blackhawks       -0.108069
Blue Jackets     -0.065312
Blues            -0.061452
Bruins           -0.073530
Canadiens        -0.064257
Canucks          -0.068376
Capitals         -0.109848
Coyotes          -0.131004
Devils           -0.118469
Ducks            -0.120684
Flames           -0.090771
Flyers           -0.182290
Golden Knights   -0.156863
Hurricanes       -0.087336
Islanders        -0.012362
Jets             -0.089766
Kings            -0.154718
Lightning        -0.129366
Maple Leafs      -0.146113
Oilers           -0.082420
Panthers         -0.103448
Penguins         -0.139920
Predators        -0.142373
Rangers          -0.054774
Red Wings        -0.033827
Sabres           -0.052402
Senators         -0.074720
Sharks           -0.081395
Stars            -0.106923
Wild             -0.131651
Name: won, dtype: float64 

small 
 teamName
Blackhawks   -0.416667
Bruins       -0.250000
Canadiens    -0.500000
Rangers      -0.500000
Red Wings 

### best_defense

In [69]:
vs_df

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,season,type,date_time,venue_time_zone_tz,teamName
0,2012030221,3,away,False,OT,John Tortorella,2,35,44,8,3,0,44.8,17,7,20122013,P,2013-05-16,EDT,Rangers
1,2012030222,3,away,False,REG,John Tortorella,2,37,33,11,5,0,51.7,1,4,20122013,P,2013-05-19,EDT,Rangers
2,2012030221,6,home,True,OT,Claude Julien,3,48,51,6,4,1,55.2,4,5,20122013,P,2013-05-16,EDT,Bruins
3,2012030222,6,home,True,REG,Claude Julien,5,32,36,19,1,0,48.3,16,6,20122013,P,2013-05-19,EDT,Bruins


In [67]:
for name, df in zip(df_names, dfs):
    print(name,"\n", df.groupby('team_id')['goals'].mean().sort_values(ascending = False).head(1), "\n")

full 
 team_id
54    3.186275
Name: goals, dtype: float64 

small 
 team_id
9    3.5
Name: goals, dtype: float64 

vs 
 team_id
6    4
Name: goals, dtype: int64 

